# SQL - NBA Portfolio

The aim of this project is to analyze NBA dataset to discover informations about each conference winner in 2022/2023 season. Besides that we will dig into some historical data to find even more interesting facts about the league.

## Importing packages and establishing connection with a database

In [192]:
import sqlite3
import pandas as pd

Let's first establish a connection with our database.

In [193]:
connection = sqlite3.connect('nba.sqlite')
cursor = connection.cursor()

## First look at the database

First thing we want to know is what tables does the database consists of.

In [194]:
pd.read_sql_query('SELECT name AS table_name FROM sqlite_master WHERE type = "table";', connection)

,table_name
0,game
1,game_summary
2,other_stats
3,officials
4,inactive_players
5,game_info
6,line_score
7,play_by_play
8,player
9,team


For the purpose of this project we will use only some of them. Below you can see the preview of tables we will use here.

In [195]:
pd.read_sql_query("SELECT * FROM game LIMIT 5;", connection)

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away
0,21946,1610610035,HUS,Toronto Huskies,0024600001,1946-11-01 00:00:00,HUS vs. NYK,L,0,25.0,...,None,None,None,None,None,None,NaN,68,2,0
1,21946,1610610034,BOM,St. Louis Bombers,0024600003,1946-11-02 00:00:00,BOM vs. PIT,W,0,20.0,...,None,None,None,None,None,None,25.0,51,-5,0
2,21946,1610610032,PRO,Providence Steamrollers,0024600002,1946-11-02 00:00:00,PRO vs. BOS,W,0,21.0,...,None,None,None,None,None,None,NaN,53,-6,0
3,21946,1610610025,CHS,Chicago Stags,0024600004,1946-11-02 00:00:00,CHS vs. NYK,W,0,21.0,...,None,None,None,None,None,None,22.0,47,-16,0
4,21946,1610610028,DEF,Detroit Falcons,0024600005,1946-11-02 00:00:00,DEF vs. WAS,L,0,10.0,...,None,None,None,None,None,None,NaN,50,17,0


In [196]:
pd.read_sql_query("SELECT * FROM game_info LIMIT 5;", connection)

,game_id,game_date,attendance,game_time
0,0024600001,1946-11-01 00:00:00,None,
1,0024600003,1946-11-02 00:00:00,None,
2,0024600002,1946-11-02 00:00:00,None,
3,0024600004,1946-11-02 00:00:00,None,
4,0024600005,1946-11-02 00:00:00,None,


In [197]:
pd.read_sql_query("SELECT * FROM play_by_play LIMIT 5;", connection)

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,0029600009,2,12,0,1,11:44 PM,12:00,None,Start of 1st Period (11:44 PM EST),None,...,None,None,0.0,0,None,None,None,None,None,0
1,0029600009,3,10,0,1,11:45 PM,12:00,Jump Ball Olajuwon vs. Polynice: Tip to Willia...,None,None,...,Kings,SAC,5.0,722,Corliss Williamson,1610612758.0,Sacramento,Kings,SAC,0
2,0029600009,4,5,4,1,11:46 PM,11:49,None,None,Williamson Traveling Turnover (P1.T1),...,None,None,0.0,0,None,None,None,None,None,0
3,0029600009,5,2,1,1,11:46 PM,11:27,MISS Olajuwon 13' Jump Shot,None,None,...,None,None,0.0,0,None,None,None,None,None,0
4,0029600009,7,4,0,1,11:46 PM,11:23,Drexler REBOUND (Off:1 Def:0),None,None,...,None,None,0.0,0,None,None,None,None,None,0


In [198]:
pd.read_sql_query("SELECT * FROM common_player_info LIMIT 5;", connection)

,person_id,first_name,last_name,display_first_last,display_last_comma_first,display_fi_last,player_slug,birthdate,school,country,...,playercode,from_year,to_year,dleague_flag,nba_flag,games_played_flag,draft_year,draft_round,draft_number,greatest_75_flag
0,51,Mahmoud,Abdul-Rauf,Mahmoud Abdul-Rauf,"Abdul-Rauf, Mahmoud",M. Abdul-Rauf,mahmoud-abdul-rauf,1969-03-09 00:00:00,Louisiana State,USA,...,mahmoud_abdul-rauf,1990.0,2000.0,N,Y,Y,1990,1,3,N
1,1505,Tariq,Abdul-Wahad,Tariq Abdul-Wahad,"Abdul-Wahad, Tariq",T. Abdul-Wahad,tariq-abdul-wahad,1974-11-03 00:00:00,San Jose State,France,...,tariq_abdul-wahad,1997.0,2003.0,N,Y,Y,1997,1,11,N
2,76007,John,Abramovic,John Abramovic,"Abramovic, John",J. Abramovic,john-abramovic,1919-02-09 00:00:00,Salem,USA,...,HISTADD_brooms_abramovic,1946.0,1947.0,N,Y,Y,Undrafted,Undrafted,Undrafted,N
3,203112,Quincy,Acy,Quincy Acy,"Acy, Quincy",Q. Acy,quincy-acy,1990-10-06 00:00:00,Baylor,USA,...,quincy_acy,2012.0,2018.0,Y,Y,Y,2012,2,37,N
4,200801,Hassan,Adams,Hassan Adams,"Adams, Hassan",H. Adams,hassan-adams,1984-06-20 00:00:00,Arizona,USA,...,hassan_adams,2006.0,2008.0,N,Y,Y,2006,2,54,N


In [199]:
pd.read_sql_query("SELECT * FROM team_details LIMIT 5;", connection)

,team_id,abbreviation,nickname,yearfounded,city,arena,arenacapacity,owner,generalmanager,headcoach,dleagueaffiliation,facebook,instagram,twitter
0,1610612737,ATL,Hawks,1949.0,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Quin Snyder,College Park Skyhawks,https://www.facebook.com/hawks,https://instagram.com/atlhawks,https://twitter.com/ATLHawks
1,1610612738,BOS,Celtics,1946.0,Boston,TD Garden,18624.0,Wyc Grousbeck,Brad Stevens,Joe Mazzulla,Maine Celtics,https://www.facebook.com/bostonceltics,https://instagram.com/celtics,https://twitter.com/celtics
2,1610612739,CLE,Cavaliers,1970.0,Cleveland,Rocket Mortgage FieldHouse,20562.0,Dan Gilbert,Koby Altman,JB Bickerstaff,Cleveland Charge,https://www.facebook.com/Cavs,https://instagram.com/cavs,https://twitter.com/cavs
3,1610612740,NOP,Pelicans,2002.0,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Willie Green,Birmingham Squadron,https://www.facebook.com/PelicansNBA,https://instagram.com/pelicansnba,https://twitter.com/PelicansNBA
4,1610612741,CHI,Bulls,1966.0,Chicago,United Center,21711.0,Jerry Reinsdorf,Arturas Karnisovas,Billy Donovan,Windy City Bulls,https://www.facebook.com/chicagobulls,https://instagram.com/chicagobulls,https://twitter.com/chicagobulls


In [200]:
pd.read_sql_query("SELECT * FROM draft_combine_stats LIMIT 5;", connection)

,season,player_id,first_name,last_name,player_name,position,height_wo_shoes,height_wo_shoes_ft_in,height_w_shoes,height_w_shoes_ft_in,...,spot_nba_break_right,spot_nba_corner_right,off_drib_fifteen_break_left,off_drib_fifteen_top_key,off_drib_fifteen_break_right,off_drib_college_break_left,off_drib_college_top_key,off_drib_college_break_right,on_move_fifteen,on_move_college
0,2000,2124,Malik,Allen,Malik Allen,PF-C,80.25,6' 8.25'',None,None,...,None,None,None,None,None,None,None,None,None,None
1,2000,12019,Harold,Arceneaux,Harold Arceneaux,SG-SF,76.50,6' 4.5'',None,None,...,None,None,None,None,None,None,None,None,None,None
2,2000,12020,Lamont,Barnes,Lamont Barnes,PF-C,80.50,6' 8.5'',None,None,...,None,None,None,None,None,None,None,None,None,None
3,2000,12131,Mario,Bland,Mario Bland,PF,77.50,6' 5.5'',None,None,...,None,None,None,None,None,None,None,None,None,None
4,2000,2056,Primoz,Brezec,Primoz Brezec,C,84.75,7' 0.75'',None,None,...,None,None,None,None,None,None,None,None,None,None


In [201]:
pd.read_sql_query("SELECT * FROM draft_history LIMIT 5;", connection)

,person_id,player_name,season,round_number,round_pick,overall_pick,draft_type,team_id,team_city,team_name,team_abbreviation,organization,organization_type,player_profile_flag
0,79299,Clifton McNeeley,1947,1,1,1,Draft,1610610031,Pittsburgh,Ironmen,PIT,Texas-El Paso,College/University,0
1,78109,Glen Selbo,1947,1,2,2,Draft,1610610035,Toronto,Huskies,HUS,Wisconsin,College/University,1
2,76649,Eddie Ehlers,1947,1,3,3,Draft,1610612738,Boston,Celtics,BOS,Purdue,College/University,1
3,79302,Walt Dropo,1947,1,4,4,Draft,1610610032,Providence,Steamrollers,PRO,Connecticut,College/University,0
4,77048,Dick Holub,1947,1,5,5,Draft,1610612752,New York,Knicks,NYK,Long Island-Brooklyn,College/University,1


In [202]:
pd.read_sql_query("SELECT * FROM team_info_common LIMIT 5;", connection)

,team_id,season_year,team_city,team_name,team_abbreviation,team_conference,team_division,team_code,team_slug,w,...,league_id,season_id,pts_rank,pts_pg,reb_rank,reb_pg,ast_rank,ast_pg,opp_pts_rank,opp_pts_pg
0,1610612737,2022-23,Atlanta,Hawks,ATL,East,Southeast,hawks,hawks,32,...,00,22022,7,117.0,12,43.7,20,24.6,21,116.9
1,1610612738,2022-23,Boston,Celtics,BOS,East,Atlantic,celtics,celtics,45,...,00,22022,4,117.7,6,45.2,7,26.3,9,112.4
2,1610612739,2022-23,Cleveland,Cavaliers,CLE,East,Central,cavaliers,cavaliers,40,...,00,22022,25,112.0,23,41.7,18,24.9,1,106.4
3,1610612740,2022-23,New Orleans,Pelicans,NOP,West,Southwest,pelicans,pelicans,31,...,00,22022,16,114.2,14,43.7,11,25.5,16,113.4
4,1610612741,2022-23,Chicago,Bulls,CHI,East,Central,bulls,bulls,29,...,00,22022,20,113.0,21,42.6,22,24.1,11,112.7


## Results of 2022/23 season

Let's receive some informations about last season from the team_info_common table combined with team table. We will order the results by conference, conference and division ranking.

In [203]:
pd.read_sql_query("SELECT team_info_common.team_city || ' ' || team_info_common.team_name AS team, team.state, team_info_common.team_conference, team_info_common.team_division, team_info_common.w AS wins, team_info_common.l AS looses, team_info_common.pct AS win_percentage, team_info_common.conf_rank,team_info_common.div_rank FROM team_info_common LEFT JOIN team ON team_info_common.team_id = team.id ORDER BY team_info_common.team_conference, team_info_common.conf_rank, team_info_common.div_rank;", connection)

,team,state,team_conference,team_division,wins,looses,win_percentage,conf_rank,div_rank
0,Milwaukee Bucks,Wisconsin,East,Central,46,18,0.719,1,1
1,Boston Celtics,Massachusetts,East,Atlantic,45,20,0.692,2,1
2,Philadelphia 76ers,Pennsylvania,East,Atlantic,41,22,0.651,3,2
3,Cleveland Cavaliers,Ohio,East,Central,40,26,0.606,4,2
4,New York Knicks,New York,East,Atlantic,39,27,0.591,5,3
5,Brooklyn Nets,New York,East,Atlantic,36,28,0.563,6,4
6,Miami Heat,Florida,East,Southeast,34,31,0.523,7,1
7,Atlanta Hawks,Atlanta,East,Southeast,32,32,0.500,8,2
8,Washington Wizards,District of Columbia,East,Southeast,30,34,0.469,10,3
9,Indiana Pacers,Indiana,East,Central,29,36,0.446,11,3


As you can see the winner in East conference was Milwaukee Bucks. In west conference best team was Denver Nuggets. Both teams have a very similar win percentage.

Teams from which division had the biggest average win percentage? Let's figure this out.

In [204]:
pd.read_sql_query('SELECT team_division, ROUND(AVG(pct),2) AS win_percentage FROM team_info_common GROUP BY team_division ORDER BY AVG(pct) DESC;', connection)

,team_division,win_percentage
0,Atlantic,0.62
1,Northwest,0.54
2,Pacific,0.53
3,Central,0.49
4,Southeast,0.44
5,Southwest,0.37


Teams from Atlantic had the biggest win percentage - 62 %.

Which teams scored the most points per game?

In [205]:
pd.read_sql_query("SELECT team_city || ' ' || team_name AS team, pts_pg AS points_per_game FROM team_info_common ORDER BY pts_pg DESC;", connection)

,team,points_per_game
0,Sacramento Kings,121.1
1,Oklahoma City Thunder,118.2
2,Golden State Warriors,118.0
3,Boston Celtics,117.7
4,Denver Nuggets,117.1
5,Atlanta Hawks,117.0
6,Los Angeles Lakers,116.7
7,Milwaukee Bucks,115.6
8,Minnesota Timberwolves,115.6
9,New York Knicks,115.5


Denver Nuggets are on fourth place here but surprisingly Milwaukee Bucks are missing in top 10. You don't need to score the best in the league to win in the finals.

Let's pose an opposite question - Which teams loosed the most points per game?

In [206]:
pd.read_sql_query("SELECT team_city || ' ' || team_name AS team, opp_pts_pg AS loosed_points_per_game FROM team_info_common ORDER BY opp_pts_pg DESC;", connection)

,team,loosed_points_per_game
0,San Antonio Spurs,122.3
1,Detroit Pistons,119.1
2,Sacramento Kings,118.5
3,Houston Rockets,118.2
4,Charlotte Hornets,117.9
5,Los Angeles Lakers,117.6
6,Indiana Pacers,117.4
7,Golden State Warriors,117.1
8,Atlanta Hawks,116.9
9,Oklahoma City Thunder,116.6


The conclusion is simple - if you want to win the league you can't lose much points.

Let's now create a view storing id of each conference team winner.

In [207]:
try :
    cursor.execute('CREATE VIEW Conference_winners AS SELECT team_id FROM team_info_common WHERE conf_rank = 1;')
except sqlite3.OperationalError:
    pass

Using previously created view now we want to get some informations about winner teams history, coach, owner and arena where they play.

In [208]:
pd.read_sql_query("SELECT city || ' ' || nickname AS team, yearfounded AS foundation_year, arena, arenacapacity, owner, headcoach FROM team_details WHERE team_id IN (SELECT * FROM Conference_winners);", connection)

,team,foundation_year,arena,arenacapacity,owner,headcoach
0,Denver Nuggets,1976.0,Ball Arena,NaN,Stan Kroenke,Michael Malone
1,Milwaukee Bucks,1968.0,Fiserv Forum,17500.0,Wesley Edens & Marc Lasry,Mike Budenholzer


## Updating missing values

Randomly we discovered that we are missing informations about Ball Arena capacity. Maybe there are more missing values?

In [209]:
pd.read_sql_query("SELECT city || ' ' || nickname AS team, arenacapacity FROM team_details WHERE arenacapacity IS NULL OR arenacapacity = 0;", connection)

,team,arenacapacity
0,New Orleans Pelicans,NaN
1,Denver Nuggets,NaN
2,Golden State Warriors,NaN
3,Brooklyn Nets,NaN
4,Orlando Magic,0.0
5,Philadelphia 76ers,NaN
6,Phoenix Suns,NaN
7,Oklahoma City Thunder,NaN
8,Toronto Raptors,NaN
9,Utah Jazz,NaN


This problem can fortunately be solved very quickly. After looking up on the Internet I found capacity of each arena. Let's update this data.

In [210]:
cursor.execute("UPDATE team_details SET arenacapacity = (CASE WHEN nickname = 'Pelicans' THEN 16867 WHEN nickname = 'Nuggets' THEN 21000 WHEN nickname = 'Warriors' THEN 18064 WHEN nickname = 'Nets' THEN 19000 WHEN nickname = '76ers' THEN 20318 WHEN nickname = 'Suns' THEN 18422 WHEN nickname = 'Thunder' THEN 13611 WHEN nickname = 'Raptors' THEN 19800 WHEN nickname = 'Jazz' THEN 15000 WHEN nickname = 'Pistons' THEN 20491 WHEN nickname = 'Magic' THEN 20000 END) WHERE nickname IN ('Pelicans', 'Nuggets', 'Warriors', 'Nets', '76ers', 'Suns', 'Thunder', 'Raptors', 'Jazz', 'Pistons', 'Magic');")

## Which team have the biggest arena?

Quick check if everything is OK now. We will display team arenas with capacity in decreasing order.

In [211]:
pd.read_sql_query("SELECT city || ' ' || nickname AS team, arena, arenacapacity FROM team_details ORDER BY arenacapacity DESC", connection)

,team,arena,arenacapacity
0,Chicago Bulls,United Center,21711.0
1,Denver Nuggets,Ball Arena,21000.0
2,Washington Wizards,Capital One Arena,20647.0
3,Cleveland Cavaliers,Rocket Mortgage FieldHouse,20562.0
4,Detroit Pistons,Little Caesars Arena,20491.0
5,Philadelphia 76ers,Wells Fargo Center,20318.0
6,Orlando Magic,Amway Center,20000.0
7,Portland Trail Blazers,Moda Center,19980.0
8,Toronto Raptors,Scotiabank Arena,19800.0
9,New York Knicks,Madison Square Garden,19763.0


The answer is - Chicago Bulls.

## Benchmarking Denver Nuggets and Milwaukee Bucks matches

Let's get the results of home and away matches of Denver Nuggets first.

In [212]:
pd.read_sql_query("SELECT team_name_away AS opponent, wl_home AS result, plus_minus_home AS plus_minus_score FROM game WHERE season_id = '22022' AND team_name_home = 'Denver Nuggets';", connection)

,opponent,result,plus_minus_score
0,Oklahoma City Thunder,W,5
1,Los Angeles Lakers,W,11
2,Utah Jazz,W,16
3,San Antonio Spurs,W,25
4,New York Knicks,L,-3
5,Detroit Pistons,L,-2
6,Houston Rockets,W,16
7,Houston Rockets,W,20
8,Dallas Mavericks,L,-1
9,Utah Jazz,W,5


In [213]:
pd.read_sql_query("SELECT team_name_home AS opponent, wl_away AS result, plus_minus_away AS plus_minus_score FROM game WHERE season_id = '22022' AND team_name_away = 'Denver Nuggets';", connection)

,opponent,result,plus_minus_score
0,Utah Jazz,L,-21
1,Golden State Warriors,W,5
2,Portland Trail Blazers,L,-25
3,Los Angeles Lakers,L,-11
4,Oklahoma City Thunder,W,12
5,San Antonio Spurs,W,6
6,Indiana Pacers,W,3
7,Boston Celtics,L,-19
8,Chicago Bulls,W,23
9,Dallas Mavericks,L,-28


Now the same thing with Milwaukee Bucks.

In [214]:
pd.read_sql_query("SELECT team_name_away AS opponent, wl_home AS result, plus_minus_home AS plus_minus_score FROM game WHERE season_id = '22022' AND team_name_home = 'Milwaukee Bucks';", connection)

,opponent,result,plus_minus_score
0,Houston Rockets,W,20
1,Brooklyn Nets,W,11
2,New York Knicks,W,11
3,Atlanta Hawks,W,8
4,Detroit Pistons,W,2
5,Detroit Pistons,W,25
6,Oklahoma City Thunder,W,14
7,Atlanta Hawks,L,-15
8,Cleveland Cavaliers,W,15
9,Portland Trail Blazers,W,8


In [215]:
pd.read_sql_query("SELECT team_name_home AS opponent, wl_away AS result , plus_minus_away AS plus_minus_score FROM game WHERE season_id = '22022' AND team_name_away = 'Milwaukee Bucks';", connection)

,opponent,result,plus_minus_score
0,Philadelphia 76ers,W,2
1,Minnesota Timberwolves,W,13
2,Atlanta Hawks,L,-19
3,Oklahoma City Thunder,W,4
4,San Antonio Spurs,L,-18
5,Philadelphia 76ers,L,-8
6,New York Knicks,W,6
7,Charlotte Hornets,W,9
8,Orlando Magic,W,7
9,Dallas Mavericks,W,1


## Fans attendance

Let's now investigate fans attendance at each home game for both teams. We will compare them with average attendance in whole season and rank particular matches with respect to this using some analytic functions. Start with Denver Nuggets.

In [216]:
pd.read_sql_query("SELECT game_info.attendance, AVG(game_info.attendance) OVER () average_attendance, rank() over (order by game_info.attendance DESC) attendance_rank FROM game JOIN game_info ON game.game_id = game_info.game_id WHERE game.season_id = '22022' AND game.team_name_home = 'Denver Nuggets';", connection)

,attendance,average_attendance,attendance_rank
0,19983,19161.194444,1
1,19896,19161.194444,2
2,19739,19161.194444,3
3,19689,19161.194444,4
4,19642,19161.194444,5
5,19641,19161.194444,6
6,19641,19161.194444,6
7,19641,19161.194444,6
8,19641,19161.194444,6
9,19638,19161.194444,10


Milwaukee Bucks

In [217]:
pd.read_sql_query("SELECT game_info.attendance, AVG(game_info.attendance) OVER () average_attendance, rank() over (order by game_info.attendance DESC) attendance_rank FROM game JOIN game_info ON game.game_id = game_info.game_id WHERE game.season_id = '22022' AND game.team_name_home = 'Milwaukee Bucks';", connection)

,attendance,average_attendance,attendance_rank
0,18100,17492.636364,1
1,18018,17492.636364,2
2,18008,17492.636364,3
3,17938,17492.636364,4
4,17713,17492.636364,5
5,17676,17492.636364,6
6,17636,17492.636364,7
7,17628,17492.636364,8
8,17627,17492.636364,9
9,17623,17492.636364,10


Arenas are almost full during both team matches.

## How does game time impact on game result?

Does the time impact on result of match for our 2 teams? Let's see the results.

First we need to convert time in format HH:MM into minutes to compare the results.

Denver at home

In [218]:
pd.read_sql_query("SELECT game.wl_home AS result, ROUND(AVG(CAST(SUBSTRING(game_info.game_time, 1, INSTR(game_info.game_time, ':') - 1) AS INT) * 60 + CAST(SUBSTRING(game_info.game_time, INSTR(game_info.game_time,':') + 1, LENGTH(game_info.game_time)) AS INT)),2) AS average_game_time FROM game JOIN game_info ON game.game_id = game_info.game_id WHERE game.season_id = '22022' AND game.team_name_home = 'Denver Nuggets' GROUP BY game.wl_home;", connection)

,result,average_game_time
0,L,137.33
1,W,137.07


Denver away

In [219]:
pd.read_sql_query("SELECT game.wl_away AS result, ROUND(AVG(CAST(SUBSTRING(game_info.game_time, 1, INSTR(game_info.game_time, ':') - 1) AS INT) * 60 + CAST(SUBSTRING(game_info.game_time, INSTR(game_info.game_time,':') + 1, LENGTH(game_info.game_time)) AS INT)),2) AS average_game_time FROM game JOIN game_info ON game.game_id = game_info.game_id WHERE game.season_id = '22022' AND game.team_name_away = 'Denver Nuggets' GROUP BY game.wl_away;", connection)

,result,average_game_time
0,L,131.19
1,W,132.69


Milwaukee home

In [220]:
pd.read_sql_query("SELECT game.wl_home AS result, ROUND(AVG(CAST(SUBSTRING(game_info.game_time, 1, INSTR(game_info.game_time, ':') - 1) AS INT) * 60 + CAST(SUBSTRING(game_info.game_time, INSTR(game_info.game_time,':') + 1, LENGTH(game_info.game_time)) AS INT)),2) AS average_game_time FROM game JOIN game_info ON game.game_id = game_info.game_id WHERE game.season_id = '22022' AND game.team_name_home = 'Milwaukee Bucks' GROUP BY game.wl_home;", connection)

,result,average_game_time
0,L,137.33
1,W,135.74


Milwaukee away

In [221]:
pd.read_sql_query("SELECT game.wl_away AS result, ROUND(AVG(CAST(SUBSTRING(game_info.game_time, 1, INSTR(game_info.game_time, ':') - 1) AS INT) * 60 + CAST(SUBSTRING(game_info.game_time, INSTR(game_info.game_time,':') + 1, LENGTH(game_info.game_time)) AS INT)),2) AS average_game_time FROM game JOIN game_info ON game.game_id = game_info.game_id WHERE game.season_id = '22022' AND game.team_name_away = 'Milwaukee Bucks' AND game.wl_away IS NOT NULL GROUP BY game.wl_home;", connection)

,result,average_game_time
0,W,140.44
1,L,136.31


Difference in results are bigger in Milwaukee Bucks. The longer they play at home the bigger is chance they will lose. On the other hand when they play longer away they are more likely to win.

## Lastest season teams' squad

Let's see players from 2022/23 season in both teams. We will calculate their age based on birthdate and mark them as veteran if they have at least 7 seasons played.

Denver Nuggets

In [222]:
pd.read_sql_query("SELECT display_first_last AS player, CAST((julianday('now') - julianday(birthdate))/365 AS INTEGER) AS age, school, country, height, weight, CASE WHEN season_exp > 6 THEN 'Yes' ELSE 'No' END AS veteranStatus, jersey, position FROM common_player_info WHERE team_code = 'nuggets' AND to_year = '2022' AND games_played_current_season_flag = 'Y';", connection)

,player,age,school,country,height,weight,veteranStatus,jersey,position
0,Bruce Brown,27,Miami,USA,6-4,202,No,11,Guard-Forward
1,Reggie Jackson,33,Boston College,USA,6-2,208,Yes,1,Guard
2,Zeke Nnaji,22,Arizona,USA,6-9,240,No,22,Forward-Center
3,Ish Smith,35,Wake Forest,USA,6-0,175,Yes,14,Guard
4,Peyton Watson,21,UCLA,USA,6-7,200,No,8,Guard
5,Jack White,26,Duke,Australia,6-7,225,No,10,Forward


Milwaukee Bucks

In [223]:
pd.read_sql_query("SELECT display_first_last AS player, CAST((julianday('now') - julianday(birthdate))/365 AS INTEGER) AS age, school, country, height, weight, CASE WHEN season_exp > 6 THEN 'Yes' ELSE 'No' END AS veteranStatus, jersey, position FROM common_player_info WHERE team_code = 'bucks' AND to_year = '2022' AND games_played_current_season_flag = 'Y';", connection)

,player,age,school,country,height,weight,veteranStatus,jersey,position
0,Grayson Allen,28,Duke,USA,6-4,198,No,12,Guard
1,Brook Lopez,35,Stanford,USA,7-1,282,Yes,11,Center
2,Wesley Matthews,37,Marquette,USA,6-5,220,Yes,23,Guard
3,Khris Middleton,32,Texas A&M,USA,6-7,222,Yes,22,Forward


## Draft picks

We are now interested about both teams draft picks from last 5 years. Let's discover players they picked.

Denver Nuggets

In [224]:
pd.read_sql_query("SELECT player_name AS player, season, round_number, round_pick, overall_pick, draft_type, organization, organization_type FROM draft_history WHERE season BETWEEN '2018' AND '2022' AND team_name = 'Nuggets';", connection)

,player,season,round_number,round_pick,overall_pick,draft_type,organization,organization_type
0,Michael Porter Jr.,2018,1,14,14,Draft,Missouri,College/University
1,Justin Jackson,2018,2,13,43,Draft,Maryland,College/University
2,Thomas Welsh,2018,2,28,58,Draft,California-Los Angeles,College/University
3,Zeke Nnaji,2020,1,22,22,Draft,Arizona,College/University
4,Bones Hyland,2021,1,26,26,Draft,Virginia Commonwealth,College/University
5,Christian Braun,2022,1,21,21,Draft,Kansas,College/University


Milwaukee Bucks

In [225]:
pd.read_sql_query("SELECT player_name AS player, season, round_number, round_pick, overall_pick, draft_type, organization, organization_type FROM draft_history WHERE season BETWEEN '2018' AND '2022' AND team_name = 'Bucks';", connection)

,player,season,round_number,round_pick,overall_pick,draft_type,organization,organization_type
0,Donte DiVincenzo,2018,1,17,17,Draft,Villanova,College/University
1,Chimezie Metu,2018,2,19,49,Draft,Southern California,College/University
2,Kevin Porter Jr.,2019,1,30,30,Draft,Southern California,College/University
3,R.J. Hampton,2020,1,24,24,Draft,Breakers (New Zealand),Other Team/Club
4,Jordan Nwora,2020,2,15,45,Draft,Louisville,College/University
5,Isaiah Todd,2021,2,1,31,Draft,Ignite (G League),Other Team/Club
6,MarJon Beauchamp,2022,1,24,24,Draft,Ignite (G League),Other Team/Club


Nuggets have choosen players only from collage or university. Bucks obtained also players from other teams.

## More details about drafted players

Let's get more details about those players. Unfortunately draft_combine_stats is not coherent with draft history table so we have some different results here.

We created a column height_quartile to assign each player height to adequate quartile.

Denver Nuggets

In [226]:
pd.read_sql_query("SELECT player_name AS player, position, height_wo_shoes AS height_without_shoes, NTILE(4) OVER (ORDER BY height_wo_shoes) AS height_quartile, height_w_shoes AS height_with_shoes, weight, wingspan, standing_reach, body_fat_pct,hand_length, hand_width, standing_vertical_leap, max_vertical_leap, lane_agility_time, three_quarter_sprint, bench_press FROM draft_combine_stats WHERE player_id IN (SELECT person_id FROM draft_history WHERE season BETWEEN '2018' AND '2022' AND team_name = 'Nuggets');", connection)

,player,position,height_without_shoes,height_quartile,height_with_shoes,weight,wingspan,standing_reach,body_fat_pct,hand_length,hand_width,standing_vertical_leap,max_vertical_leap,lane_agility_time,three_quarter_sprint,bench_press
0,Nah'Shon Hyland,SG,74.00,1,75.50,169.00,81.25,101.5,5.8,8.75,9.0,26.5,34.0,10.53,3.10,NaN
1,Christian Braun,SG,77.50,1,79.00,209.20,78.50,99.5,6.2,8.0,8.5,33.5,40.0,11.00,3.16,NaN
2,Justin Jackson (UMD),SF,77.75,2,79.00,219.4,87.25,107.0,7.1,9.0,9.5,26.5,31.5,11.78,3.43,NaN
3,Justin Jackson,PF,78.00,2,78.75,229.4,87.00,105.0,8.9,9.0,9.5,NaN,NaN,NaN,NaN,NaN
4,Zeke Nnaji,PF-C,81.25,3,82.50,246.8,86.00,105.5,5.9,9.0,10.5,33.0,38.0,10.94,3.20,NaN
5,Michael Porter,SF-PF,81.50,3,82.75,211,84.25,108.5,6.4,8.75,9.25,NaN,NaN,NaN,NaN,NaN
6,Thomas Welsh,PF-C,83.50,4,84.50,254,84.00,109.5,10.9,9.0,10.5,NaN,NaN,NaN,NaN,11.0


Milwaukee Bucks

In [227]:
pd.read_sql_query("SELECT player_name AS player, position, height_wo_shoes AS height_without_shoes, NTILE(4) OVER (ORDER BY height_wo_shoes) AS height_quartile, height_w_shoes AS height_with_shoes, weight, wingspan, standing_reach, body_fat_pct,hand_length, hand_width, standing_vertical_leap, max_vertical_leap, lane_agility_time, three_quarter_sprint, bench_press FROM draft_combine_stats WHERE player_id IN (SELECT person_id FROM draft_history WHERE season BETWEEN '2018' AND '2022' AND team_name = 'Bucks');", connection)

,player,position,height_without_shoes,height_quartile,height_with_shoes,weight,wingspan,standing_reach,body_fat_pct,hand_length,hand_width,standing_vertical_leap,max_vertical_leap,lane_agility_time,three_quarter_sprint,bench_press
0,Donte DiVincenzo,SG,75.50,1,76.5,200.6,78.00,97.5,5.0,8.25,9.0,34.5,42.0,10.72,3.11,12.0
1,Kevin Porter Jr.,SG,76.00,1,77.5,212.6,81.00,103.0,5.1,8.75,8.75,27.0,34.0,10.82,3.14,2.0
2,MarJon Beauchamp,SG,77.25,2,78.5,196.60,84.75,106.0,5.3,9.0,9.0,NaN,NaN,NaN,NaN,NaN
3,Jordan Nwora,PF,77.75,2,79.5,220,82.50,103.0,7.8,8.5,9.25,NaN,NaN,NaN,NaN,NaN
4,Jordan Nwora,SF-PF,77.75,3,79.0,223,82.25,104.0,11.68,8.5,9.25,29.0,35.0,11.29,3.43,NaN
5,Chimezie Metu,PF-C,80.50,3,81.5,219.6,84.50,108.0,5.05,9.0,9.25,29.5,36.0,11.89,3.33,5.0
6,Isaiah Todd,PF,80.75,4,82.0,219.00,85.25,107.5,6.2,9.0,9.75,28.5,35.5,11.38,3.31,NaN


## Pick numbers

How many high picks (overall player pick lower than 10) does both teams made in the history?

Denver Nuggets

In [228]:
pd.read_sql_query("SELECT COUNT(*) AS number_of_high_picks FROM draft_history WHERE overall_pick < 10 AND team_name = 'Nuggets';", connection)

,number_of_high_picks
0,13


Milwaukee Bucks

In [229]:
pd.read_sql_query("SELECT COUNT(*) AS number_of_high_picks FROM draft_history WHERE overall_pick < 10 AND team_name = 'Bucks';", connection)

,number_of_high_picks
0,16


## Highest player in the league history

I think everyone is curious who was the tallest player in the NBA. The answer is below.

In [230]:
pd.read_sql_query("SELECT display_first_last AS player, team_city || ' ' || team_name AS team, CAST((julianday('now') - julianday(birthdate))/365 AS INTEGER) AS age, school, country, MAX(height) AS height, weight, CASE WHEN season_exp > 6 THEN 'Yes' ELSE 'No' END AS veteranStatus, jersey, position FROM common_player_info;", connection)

,player,team,age,school,country,height,weight,veteranStatus,jersey,position
0,Gheorghe Muresan,Washington Bullets,52,Pau Orthez,Romania,7-7,303,Yes,77,Center


That's almost 2,31 m !

## National minorities in the NBA

The fact that NBA is dominated by Americans is obvious. We are more interested in what nationalities are in the next places in terms of the number of players in the league. We will use a window function to present what percent of all players represent particular nationality.

In [231]:
pd.read_sql_query("SELECT country, COUNT(DISTINCT(person_id)), 100.0*COUNT(DISTINCT(person_id))/SUM(COUNT(DISTINCT(person_id))) OVER () AS percent_of_all_players FROM common_player_info GROUP BY country ORDER BY 2 DESC;", connection)

,country,COUNT(DISTINCT(person_id)),percent_of_all_players
0,USA,2221,90.394790
1,Canada,26,1.058201
2,France,22,0.895401
3,Serbia,18,0.732601
4,Croatia,12,0.488400
5,Australia,10,0.407000
6,Germany,10,0.407000
7,Argentina,8,0.325600
8,Lithuania,7,0.284900
9,Turkey,7,0.284900


Canada is on the second place here. The first european country in this ranking is France with 22 players.

## Most dunks in one game

Have you ever wondered what is the biggest number of dunks in a single game? Let's find out the answer. We just need to search for pattern "dunk" in a single match description and count them.

In [232]:
pd.read_sql_query("SELECT COUNT(*) AS number_of_dunks FROM play_by_play WHERE LOWER(homedescription) LIKE '%dunk%' GROUP BY game_id ORDER BY 1 DESC LIMIT 1;", connection)

,number_of_dunks
0,21


Woow 21 dunks - that's a lot. Imagine how spectacular and at the same time one-horned this match must have been !

# Summary

We have disovered a lot of interesting informations about NBA. Keep in mind that we have used only a small part of the dataset. Feel free to explore them on your own!